In [3]:
%load_ext autoreload
%autoreload 2


In [4]:
# Train the ML model
from utils import sample_files, encoder_files_to_tensors
from utils import plot_loss, load_encoder_data
from models import Encoder

import time
import glob
import tensorflow as tf
from tensorflow import keras
import yaml
import os
import numpy as np
from datetime import datetime
import argparse
import matplotlib.pyplot as plt
import matplotlib as mpl
num_Turns_Case = 1


In [5]:
# Initialize parameters
data_dir = './tomo_data/datasets_encoder_02-12-22'

timestamp = datetime.now().strftime("%Y_%m_%d_%H-%M-%S")
print('Using timestamp: ', timestamp)



Using timestamp:  2022_12_16_09-45-54


In [6]:
# Data specific
IMG_OUTPUT_SIZE = 128
# BUFFER_SIZE = 256
BATCH_SIZE = 32  # 8
latent_dim = 7  # 6 + the new VrfSPS
# additional_latent_dim = 1

# Train specific
train_cfg = {
    'epochs': 60,
    'dense_layers': [64, latent_dim],
    'filters': [8, 16, 32],
    'cropping': [0, 0],
    'kernel_size': [5, 5, 5],
    'strides': [2, 2],
    'activation': 'relu',
    'pooling': None,
    'pooling_size': [0, 0],
    'pooling_strides': [1, 1],
    'pooling_padding': 'valid',
    'dropout': 0.,
    'loss': 'mse',
    'lr': 1e-3,
    'dataset%': 1,
    'normalization': 'minmax',
    'loss_weights': [1, 1, 1, 1, 1, 1, 1],
    'batch_size': 32
}



In [ ]:
# Initialize directories
trial_dir = os.path.join('./trials/', timestamp)
weights_dir = os.path.join(trial_dir, 'weights')
plots_dir = os.path.join(trial_dir, 'plots')

# Initialize train/ test / validation paths
ML_dir = os.path.join(data_dir, 'ML_data')
TRAINING_PATH = os.path.join(ML_dir, 'TRAINING')
VALIDATION_PATH = os.path.join(ML_dir, 'VALIDATION')
assert os.path.exists(TRAINING_PATH)
assert os.path.exists(VALIDATION_PATH)

# create the directory to store the results
os.makedirs(trial_dir, exist_ok=True)
os.makedirs(weights_dir, exist_ok=False)
os.makedirs(plots_dir, exist_ok=False)

# Initialize GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
device_to_use = 0

if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        tf.config.experimental.set_memory_growth(gpus[device_to_use], True)
        tf.config.experimental.set_virtual_device_configuration(
            gpus[device_to_use],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=12*1024)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(
            logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
else:
    print('No GPU available, using the CPU')


In [ ]:
start_t = time.time()
# Create the datasets
# 1. Randomly select the training data
file_names = sample_files(
    TRAINING_PATH, train_cfg['dataset%'], keep_every=num_Turns_Case)
print('Number of Training files: ', len(file_names))

x_train, y_train = encoder_files_to_tensors(
    file_names, normalization=train_cfg['normalization'])

# Repeat for validation data
file_names = sample_files(
    VALIDATION_PATH, train_cfg['dataset%'], keep_every=num_Turns_Case)
print('Number of Validation files: ', len(file_names))

x_valid, y_valid = encoder_files_to_tensors(
    file_names, normalization=train_cfg['normalization'])

end_t = time.time()
print(
    f'\n---- Input files have been read, elapsed: {end_t - start_t} ----\n')


In [ ]:
%matplotlib inline
# plot some of the outputs

nrows = 3
# Get nrows * nrows random images
sample = np.random.choice(np.arange(len(x_train)),
                          size=nrows * nrows, replace=False)

samples_X = tf.gather(x_train, sample)
samples_y = tf.gather(y_train, sample)

# Create 3x3 grid of figures
fig, axes = plt.subplots(ncols=nrows, nrows=nrows, figsize=(12, 12))
axes = np.ravel(axes)
for i in range(len(axes)):
    ax = axes[i]
    ax.set_xticks([])
    ax.set_yticks([])
    # show the image
    ax.imshow(samples_X[i], cmap='jet')
    # Set the label
    title = ','.join([f'{num:.1f}' for num in samples_y[i]])
    ax.set_title(f'{title}')


In [ ]:
# Model instantiation

input_shape = (IMG_OUTPUT_SIZE, IMG_OUTPUT_SIZE, 1)

encoder = Encoder(input_shape=input_shape, **train_cfg)

print(encoder.model.summary())



In [ ]:
# Train the encoder

# callbacks, save the best model, and early stop if no improvement in val_loss
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss',
                                           patience=5, restore_best_weights=True)
save_best = keras.callbacks.ModelCheckpoint(filepath=os.path.join(weights_dir, 'encoder.h5'),
                                            monitor='val_loss', save_best_only=True)


In [ ]:
start_time = time.time()
# history = encoder.model.fit(
#     train_dataset, epochs=train_cfg['epochs'],
#     validation_data=valid_dataset,
#     callbacks=[stop_early, save_best])
history = encoder.model.fit(
    x_train, y_train, epochs=train_cfg['epochs'],
    validation_data=(x_valid, y_valid), 
    batch_size=train_cfg['batch_size'],
    callbacks=[save_best])

total_time = time.time() - start_time


In [ ]:
print(history.history)

In [ ]:
from sklearn.metrics import mean_absolute_error

from functools import partial


def my_mse(y_true, y_pred):
    return tf.reduce_mean(tf.math.abs(y_pred - y_true), axis=0)


y_true = np.array([[0., 0.], [1., 1.]])
y_pred = np.array([[1., 0.5], [2., 1.5]])
loss = keras.losses.mean_absolute_error(y_true, y_pred)
mae2 = MyMeanAbsoluteError(reduction=keras.losses.Reduction.NONE)
mae = keras.losses.MeanAbsoluteError(reduction=keras.losses.Reduction.NONE)
lossv2 = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
# lossv3 = my_mse(1, y_true, y_pred)
# print(loss)
# print(mae(y_true, y_pred).numpy())
print(lossv2)
# print(mae2(y_true, y_pred).numpy())
# lossv4 = my_mse(y_true, y_pred)

print(my_mse(y_true, y_pred))
# print(my_mse(1, y_true, y_pred))


In [ ]:
# Plot training and validation loss
train_loss_l = np.array(history.history['loss'])
valid_loss_l = np.array(history.history['val_loss'])

plot_loss({'Training': train_loss_l, 'Validation': valid_loss_l},
          title='Encoder Train/Validation Loss',
          figname=os.path.join(plots_dir, 'encoder_train_valid_loss.png'))


In [ ]:
from sklearn.metrics import mean_squared_error

# get predictions
y_pred = encoder.model.predict(x_valid, verbose=False)
y_valid = np.array(y_valid)

# Calculate error per variable
mses = mean_squared_error(y_valid, y_pred, multioutput='raw_values')

var_names = ['phase_error', 'energy_error',
             'bunch_length', 'intensity', 'Volt_rf', 'mu', 'Vrf_SPS']
# report
print('Variable\tMSE')
for name, mse in zip(var_names, mses):
    print(f'{name}:\t{mse:.4f}')


In [ ]:
# save file with experiment configuration
config_dict = {}
config_dict['encoder'] = train_cfg.copy()
config_dict['encoder'].update({
    'min_train_loss': float(np.min(train_loss_l)),
    'min_valid_loss': float(np.min(valid_loss_l)),
    'total_train_time': total_time,
    'used_gpus': len(gpus)
})

# save config_dict
with open(os.path.join(trial_dir, 'encoder-summary.yml'), 'w') as configfile:
    yaml.dump(config_dict, configfile, default_flow_style=False)
